In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download borhanitrash/alzheimer-mri-disease-classification-dataset

In [ ]:
# import zipfile

# zip_ref=zipfile.ZipFile('/content/Dataset.zip','r')
# zip_ref.extractall('/Dataset.zip')
# zip_ref.close()

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install keras
!pip install tensorflow

In [ ]:
#Set up the environment and upload the data
from sklearn.preprocessing import StandardScaler
from  glob import glob
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
from skimage.transform import resize
import pandas as pd
from matplotlib.image import imread
from skimage.io import imread_collection
from PIL import Image
import seaborn as sns
from sklearn import decomposition, preprocessing, svm
import sklearn.metrics as metrics #confusion_matrix, accuracy_score
from time import sleep
from tqdm.notebook import tqdm
import os
sns.set()

In [ ]:
!pip install numpy==1.26.4 --force-reinstall


In [ ]:
import os
os.kill(os.getpid(), 9)



In [ ]:
import numpy as np
print(np.__version__)  # should print 1.26.4


1.26.4


In [ ]:
import zipfile

zip_path = '/content/Dataset.zip'

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(f"✅ ZIP is valid. Found {len(file_list)} files.")
        print("First 5 files:")
        print(file_list[:5])
except zipfile.BadZipFile:
    print("❌ BadZipFile: This is not a valid ZIP file. Re-upload or re-download it.")


✅ ZIP is valid. Found 12812 files.
First 5 files:
['Dataset/', '__MACOSX/._Dataset', 'Dataset/.DS_Store', '__MACOSX/Dataset/._.DS_Store', 'Dataset/Moderate_Demented/']


In [ ]:
import zipfile

zip_path = '/content/Dataset.zip'
extract_path = '/content'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted to /content")


✅ Dataset extracted to /content


In [ ]:
import glob

# Dataset that should go with Alzheimer label
very_mild = glob.glob(r'/content/Dataset/Very_Mild_Demented/*')
mild = glob.glob(r'/content/Dataset/Mild_Demented/*')
moderate = glob.glob(r'/content/Dataset/Moderate_Demented/*')

# Dataset without Alzheimer
non = glob.glob(r'/content/Dataset/Non_Demented/*')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def view_image(directory):
    img = mpimg.imread(directory)
    plt.imshow(img)
    plt.title(directory)
    plt.axis('off')
    print(f'Image shape: {img.shape}')
    return img

print('One of the data in Non Alzheimer Folder')
print(non[2])
view_image(non[2])


In [ ]:
import matplotlib.image as mpimg

In [ ]:
print(non[2])
import matplotlib.image as mpimg # Import the necessary module

def view_image(directory):
    img = mpimg.imread(directory)
    plt.imshow(img)
    plt.title(directory)
    plt.axis('off')
    print(f'Image shape:{img.shape}')
    return img

print('One of the data in Non Alzheimer Folder')
view_image(non[2])

Resizing of data


In [ ]:
from PIL import Image
import numpy as np

# List where arrays shall be stored
resized_image_array = []
resized_image_array_label = []

width = 256
height = 256
new_size = (width, height)

# Iterate over pictures and resize them
def resizer(image_directory):
    for file in image_directory:
        img = Image.open(file)
        img = img.resize(new_size)
        array_temp = np.array(img)
        shape_new = width * height
        img_wide = array_temp.reshape(1, shape_new)
        resized_image_array.append(img_wide[0])
        if image_directory == non:
            resized_image_array_label.append(0)
        else:
            resized_image_array_label.append(1)

ALZ = very_mild + mild + moderate
resizer(non)
resizer(ALZ)


In [ ]:
!pip install scipy==1.12.0 --force-reinstall


In [ ]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split


# Assuming these are defined earlier
resized_image_array = []
resized_image_array_label = []

# Example input: non and ALZ contain lists of image file paths
def resizer(image_directory, label):
    for file in image_directory:
        img = Image.open(file).convert('L')  # Convert to grayscale
        img = img.resize((256, 256))         # Resize to 256x256
        img_array = np.array(img, dtype=np.float32).flatten()  # Convert and flatten
        resized_image_array.append(img_array)
        resized_image_array_label.append(label)

# Process images for each category
resizer(non, 0)   # Label 0 for non-ALZ
resizer(ALZ, 1)   # Label 1 for ALZ

# Convert to numpy arrays
resized_image_array = np.array(resized_image_array, dtype=np.float32)
resized_image_array_label = np.array(resized_image_array_label, dtype=np.int32)

# Optional: check shapes
print("Image array shape:", resized_image_array.shape)
print("Label array shape:", resized_image_array_label.shape)

# Split the data
train_x, test_x, train_y, test_y = train_test_split(
    resized_image_array,
    resized_image_array_label,
    test_size=0.2,
    random_state=42  # For reproducibility
)



In [ ]:
!pip install scikit-learn==1.6.1 --force-reinstall



In [ ]:


!pip install numpy==1.26.4 --force-reinstall


In [ ]:
import numpy
print(numpy.__version__)


1.26.4


In [ ]:
!pip uninstall -y scikit-image
!pip install scikit-image==0.19.3

feature extraction using GLCM

In [ ]:
import numpy as np
from skimage.feature import graycomatrix, graycoprops

# GLCM parameters
distances = [1]
angles = [0]
properties = ['energy', 'contrast', 'correlation', 'homogeneity', 'ASM']

def preprocess_image(image, width, height):
    """Ensure the image is uint8 and shaped correctly."""
    image = image.reshape((width, height))  # reshape if needed
    if image.dtype != np.uint8:
        if image.max() <= 1.0:
            image = (image * 255).astype(np.uint8)
        else:
            image = image.astype(np.uint8)
    return image

def extract_glcm_features(images, width, height):
    features_list = []
    for img in images:
        img = preprocess_image(img, width, height)
        glcm = graycomatrix(img, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
        features = [graycoprops(glcm, prop)[0][0] for prop in properties]
        features_list.append(features)
    return features_list

# Set correct image dimensions
width, height = 256, 256  # update to match your actual resized image size

# Apply feature extraction
train_glcm_features = extract_glcm_features(train_x, width, height)
test_glcm_features = extract_glcm_features(test_x, width, height)



In [ ]:
!pip uninstall -y scipy
!pip install scipy --upgrade --force-reinstall


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# create an SVM classifier with a linear kernel
clf = SVC(kernel='linear')

# fit the classifier to the training data
clf.fit(train_glcm_features, train_y)

# make predictions on the test data
y_pred = clf.predict(test_glcm_features)

# compute the accuracy of the classifier
accuracy = accuracy_score(test_y, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.62109375


In [ ]:
# import numpy as np
# from PIL import Image
# from skimage.feature import greycomatrix, greycoprops
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# # Load and preprocess the new image
# def preprocess_new_image(img_path, new_size=(256, 256)):
#     img = Image.open(img_path)
#     img = img.resize(new_size)
#     img_array = np.array(img)
#     return img_array

# # Extract GLCM features from the image
# # def extract_glcm_features(img_array, distances=[1], angles=[0], properties=['energy', 'contrast', 'correlation', 'homogeneity', 'ASM']):
# #     glcm = greycomatrix(img_array, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
# #     features = [greycoprops(glcm, prop)[0][0] for prop in properties]
# #     return features

# def extract_glcm_features(img_array, distances=[1], angles=[0], properties=['energy', 'contrast', 'correlation', 'homogeneity', 'ASM']):
#     glcm = greycomatrix(img_array, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
#     features = [greycoprops(glcm, prop)[0][0] for prop in properties]
#     return features

# # Function to predict if the person has dementia
# def predict_dementia(img_path, clf, new_size=(256, 256)):
#     img_array = preprocess_new_image(img_path, new_size)
#     features = extract_glcm_features(img_array)
#     prediction = clf.predict([features])
#     if prediction[0] == 1:
#         return 'Dementia'
#     else:
#         return 'No Dementia'
# img_path = '/content/dementedeimage.jpeg' # Replace with the path to your new MRI image
# result = predict_dementia(img_path, clf)
# print(f'The prediction for the MRI image is: {result}')

In [ ]:
!pip uninstall -y scikit-image
!pip install scikit-image==0.21.0


In [ ]:
import numpy as np
from skimage.feature import graycomatrix
from skimage.color import rgb2gray
from PIL import Image

# Load and preprocess image
img = Image.open("/content/Dataset/Non_Demented/non_577.jpg").convert('L')
gray_img = np.array(img)

# Compute GLCM
glcm = graycomatrix(gray_img, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
glcm = glcm[:, :, 0, 0]  # Extract 2D matrix

# --- Manually compute texture features ---
# 1. Contrast
i, j = np.indices(glcm.shape)
contrast = np.sum(glcm * (i - j) ** 2)

# 2. Energy
energy = np.sum(glcm ** 2)

# 3. Homogeneity
homogeneity = np.sum(glcm / (1.0 + np.abs(i - j)))

# 4. ASM (Angular Second Moment)
asm = energy  # same as energy

# 5. Dissimilarity
dissimilarity = np.sum(glcm * np.abs(i - j))

# Output
print(f"Contrast: {contrast:.4f}")
print(f"Energy: {energy:.4f}")
print(f"Homogeneity: {homogeneity:.4f}")
print(f"Dissimilarity: {dissimilarity:.4f}")



Contrast: 264.1829
Energy: 0.2065
Homogeneity: 0.5587
Dissimilarity: 8.2811


In [ ]:
import numpy as np
from PIL import Image
from skimage.feature import graycomatrix, greycoprops
from skimage.color import rgb2gray
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
def preprocess_new_image(img_path, new_size=(256, 256)):
    img = Image.open(img_path).convert('RGB')      # Ensure it's RGB
    img = img.resize(new_size)
    img_array = np.array(img)
    gray_image = rgb2gray(img_array)               # Convert to grayscale
    gray_image = (gray_image * 255).astype(np.uint8)  # Convert to uint8 (0-255)
    return gray_image


In [ ]:
def extract_glcm_features(img_array, distances=[1], angles=[0], properties=['energy', 'contrast', 'correlation', 'homogeneity', 'ASM']):
    glcm = graycomatrix(img_array, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    features = [greycoprops(glcm, prop)[0][0] for prop in properties]
    return features


In [ ]:
def predict_dementia(img_path, clf, new_size=(256, 256)):
    img_array = preprocess_new_image(img_path, new_size)
    features = extract_glcm_features(img_array)
    prediction = clf.predict([features])
    return 'Dementia' if prediction[0] == 1 else 'No Dementia'


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
img_path = 'dementedimage.jpeg'  # No need for /content/ if it's uploaded via Colab
result = predict_dementia(img_path, clf);
print(f'The prediction for the MRI image is: {result}')



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# create a RF classifier with 100 trees
clf = RandomForestClassifier(n_estimators=100)

# fit the classifier to the training data
clf.fit(train_glcm_features, train_y)

# make predictions on the test data
y_pred = clf.predict(test_glcm_features)

# compute the accuracy of the classifier
accuracy = accuracy_score(test_y, y_pred)

print("Accuracy:", accuracy)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
# perform k-fold cross-validation (here k=5)
scores = cross_val_score(clf, train_glcm_features, train_y, cv=5)

# print the accuracy for each fold
print("Cross-validation scores:", scores)

# compute the mean and standard deviation of the accuracy across all folds
mean_accuracy = scores.mean()
std_accuracy = scores.std()

print("Mean accuracy:", mean_accuracy)
print("Standard deviation:", std_accuracy)

feature extraction using Gabor Filter

In [ ]:
import cv2
# Define the parameters for the Gabor filter
ksize = 31  # size of the filter kernel
sigma = 4  # standard deviation of the Gaussian function used in the filter
theta = 0  # orientation of the filter
lambd = 10  # wavelength of the sinusoidal factor in the filter
gamma = 0.5  # spatial aspect ratio of the filter

# Create the Gabor filter
kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, 0, ktype=cv2.CV_32F)

# Apply the Gabor filter to each image
gabor_features_train = []
for img in train_x:
    # Convert the image from a 1D array to a 2D array
    img_2d = np.reshape(img, (height, width)).astype(np.float32)
    # Apply the Gabor filter to the image
    filtered_img = cv2.filter2D(img_2d, cv2.CV_8UC3, kernel)
    # Convert the filtered image back to a 1D array and add it to the feature list
    gabor_features_train.append(np.ravel(filtered_img))

gabor_features_test = []
for img in test_x:
    # Convert the image from a 1D array to a 2D array
    img_2d = np.reshape(img, (height, width)).astype(np.float32)
    # Apply the Gabor filter to the image
    filtered_img = cv2.filter2D(img_2d, cv2.CV_8UC3, kernel)
    # Convert the filtered image back to a 1D array and add it to the feature list
    gabor_features_test.append(np.ravel(filtered_img))


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
img = cv2.imread('/content/Dataset/Non_Demented/non.jpg', 0)

# Define parameters for Gabor filter
ksize = 31
sigma = 5
theta = 0
lamda = 10
gamma = 0.5
phi = 0

# Create Gabor filter
kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)

# Apply Gabor filter to image
filtered_img = cv2.filter2D(img, cv2.CV_8UC3, kernel)

# Display original image and filtered image side by side
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img, cmap='gray')
ax[0].set_title('Original Image')
ax[0].axis('off')
ax[1].imshow(filtered_img, cmap='gray')
ax[1].set_title('Filtered Image')
ax[1].axis('off')
plt.show()



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# create a RF classifier with 100 trees
clf = RandomForestClassifier(n_estimators=100)

# fit the classifier to the training data
clf.fit(gabor_features_train, train_y)

# make predictions on the test data
y_pred = clf.predict(gabor_features_test)

# compute the accuracy of the classifier
accuracy = accuracy_score(test_y, y_pred)

print("Accuracy:", accuracy)

In [ ]:
# Add Linear Regression model (Logistic Regression)
from sklearn.linear_model import LogisticRegression

# Create a logistic regression classifier
clf_lr = LogisticRegression(max_iter=1000)

# Fit the classifier to the training data
clf_lr.fit(train_glcm_features, train_y)

# Make predictions on the test data
y_pred_lr = clf_lr.predict(test_glcm_features)

# Compute the accuracy of the classifier
accuracy_lr = accuracy_score(test_y, y_pred_lr)

# Print the accuracy
print("Logistic Regression Accuracy:", accuracy_lr)


In [ ]:
# import matplotlib.pyplot as plt

# # Accuracies of the models
# accuracies = {
#     'SVM': accuracy,
#     'Random Forest': accuracy_r,
#     'Logistic Regression': accuracy_lr
# }

# # Plotting the accuracies
# plt.figure(figsize=(10, 5))
# plt.bar(accuracies.keys(), accuracies.values(), color=['blue', 'green', 'red'])
# plt.xlabel('Model')
# plt.ylabel('Accuracy')
# plt.title('Accuracies of Different Models')
# plt.ylim(0, 1)  # Accuracy ranges from 0 to 1
# plt.show()


mean square error of RF